In [1]:
%load_ext autoreload
%autoreload 2

# Multicollinearity | All Variables

In [2]:
import sys
sys.path.append('../')
import tokamakTK

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sp
import statsmodels.api as sm
import matplotlib.patches as mpatches
import matplotlib.colors as mcolors
import seaborn as sns
import plotly.express as px
import plotly.subplots as plsp
import plotly.graph_objects as go

from sklearn.decomposition import PCA
from plotly.subplots import make_subplots
from tokamakTK import MyCounter, HUEOrder
from collections import Counter
from statsmodels.stats.outliers_influence import variance_inflation_factor


pd.set_option('display.max_columns', None)
plt.rc('font',family = 'serif')
TD_colors = plt.cm.get_cmap('flag', 20) #Paired, flag
colors_ = sns.color_palette('viridis', 20)
CSS_colors = [mcolors.to_hex(TD_colors(i)) for i in range(20)]
path = "../data/"

/tmp/ipykernel_5135/4012704369.py:26: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  TD_colors = plt.cm.get_cmap('flag', 20) #Paired, flag


In [3]:
# Obtained from Optimization
min_subset_ids = pd.read_csv(path+"R_ids_alpha_0.6357.csv")

DB2 = pd.read_csv(path+"DB2P8.csv")
DB5 = pd.read_csv(path+"SELDB5_SVD.csv", low_memory=False) 

# Setting ELMy Dataset
DB5 = DB5[DB5["PHASE"].isin(['HGELM', 'HSELM', 'HGELMH', 'HSELMH'])]

# REMOVING SPHERICAL TOKAMAKS
#DB5 = DB5[~DB5.TOK.isin(['MAST', 'NSTX', 'START'])]

# There is two shots from DB2P8 missing in DB5
missing_shots = DB2[~DB2.id.isin( DB5.id.values )].reset_index(drop=True)
DB5 = pd.concat([DB5, missing_shots], axis=0, ignore_index=True)

# Labeling shots that had great impact in decreasing alpha_R
DB5.insert(loc=2,column="label",value=[0]*len(DB5))
DB5.insert(loc=2,column="label_str",value=["Unaffected"]*len(DB5))
DB5.loc[(DB5[DB5.id.isin(min_subset_ids.id)].index), "label"] = 1
DB5.loc[(DB5[DB5.id.isin(min_subset_ids.id)].index), "label_str"] = "Decreasing"

print(
    f"{ round( (len(min_subset_ids)/len(DB5))*100     ,2)  }% of the data decreased alpha_R\n" + 
    f"{ round( (1 - len(min_subset_ids)/len(DB5))*100 ,2)  }% of the data did not decrease alpha_R"
)

23.45% of the data decreased alpha_R
76.55% of the data did not decrease alpha_R


```Python 

plasma_characteristics = ["QCYL5","BEIMHD","PREMAG","LHTIME","HYBRID",
                          "CONFIG","DWDIA","WMHD","TORQ","KAREA", "EPS","MEFF","VOL","LCOULOMB",
                          "IP","RHOSTAR","NUSTAR","BETASTAR"] 
TOK_characteristics = ["TOK","DIVNAME","WALMAT","DIVMAT","LIMMAT","AMIN","BT"]
ELM = ["ELMTYPE","ELMFREQ"]
heating = ["PECRH", "PICRH", "ICSCHEME","AUXHEAT","ECHMODE","PELLET"]
impurities = ["EVAP","ZEFF","ZEFFNEO","PRAD","POHM","ENBI","PNBI"]
power = ["PLTH","PFLOSS"]
temperatures = ["TAV","TEV","TIV"]
fast_particles = ["NESOL","WFFORM","WFICFORM","OMEGACYCL","NEL"] 

# -----------------------------------------------------------------------------------
# One subset found with low multicollinearity and appeared to perform well with GP
features = ['NUSTAR', 'BETASTAR', 'QCYL5', 'HYBRID', 'ENBI', 'VOL', 'POHM',
            'PNBI', 'DWDIA', 'BT', 'NEL', 'PFLOSS', 'KAREA', 'MEFF', 'WFICFORM', 'IP']

>>> precision_recall_fscore_support(y_test, y_pred_, labels=[1,0]) | GP
(array([0.84552846, 0.92896936]),
 array([0.75362319, 0.95902229]),
 array([0.79693487, 0.94375663]),
 array([ 414, 1391]))


n_estimators = 290; max_depth = 22
>>> precision_recall_fscore_support(y_test, y_pred_, labels=[1,0]) | RF
(array([0.87301587, 0.94113525]),
 array([0.79710145, 0.96549245]),
 array([0.83333333, 0.95315827]),
 array([ 414, 1391]))

# -----------------------------------------------------------------------------------
# Subset with VIF < 10
features = ['AUXHEAT','BEIMHD','BETASTAR','BT','CONFIG','DIVMAT','DIVNAME','DWDIA',
            'ECHMODE','ELMFREQ','ELMTYPE','ENBI','EPS','EVAP','HYBRID','ICSCHEME','IP',
            'KAREA','LHTIME','LIMMAT','MEFF','NEL','NESOL','NUSTAR','OMEGACYCL',
            'PECRH','PELLET','PFLOSS','PICRH','PNBI','POHM','PRAD','PREMAG','QCYL5',
            'RHOSTAR','TAV','TIV','TOK','TORQ','WALMAT','WFFORM','WFICFORM',
            'ZEFF','ZEFFNEO']
>>> precision_recall_fscore_support(y_test, y_pred_, labels=[1,0]) | GP
(array([0.82692308, 0.92158223]),
 array([0.72705314, 0.95470884]),
 array([0.77377892, 0.93785311]),
 array([ 414, 1391]))


# -----------------------------------------------------------------------------------
# Susbet with all features having a VIF < 5
features = ['AUXHEAT','BEIMHD','BT','CONFIG','DIVMAT','DIVNAME','DWDIA',
            'ECHMODE','ELMFREQ','ELMTYPE','EPS','EVAP','HYBRID','ICSCHEME','IP',
            'KAREA','LHTIME','LIMMAT','MEFF','NEL','NESOL','NUSTAR',
            'PECRH','PELLET','PFLOSS','PICRH','PNBI','POHM','PREMAG','QCYL5',
            'RHOSTAR','TAV','TIV','TOK','TORQ','WALMAT','WFFORM','WFICFORM',
            'ZEFFNEO'] 

>>> precision_recall_fscore_support(y_test, y_pred_, labels=[1,0]) | GP
(array([0.85260116, 0.91843729]),
 array([0.71256039, 0.96333573]),
 array([0.77631579, 0.94035088]),
 array([ 414, 1391]))

```

In [121]:
features = ['AUXHEAT','BEIMHD','BT','CONFIG','DIVMAT','DIVNAME','DWDIA',
            'ECHMODE','ELMFREQ','ELMTYPE','EPS','EVAP','HYBRID','ICSCHEME','IP',
            'KAREA','LHTIME','LIMMAT','MEFF','NEL','NESOL','NUSTAR',
            'PECRH','PELLET','PFLOSS','PICRH','PNBI','POHM','PREMAG','QCYL5',
            'RHOSTAR','TAV','TIV','TOK','TORQ','WALMAT','WFFORM','WFICFORM',
            'ZEFFNEO'] 

# Removing features with high VIF:
# AMIN, PLTH, TEV, VOL, LCOULOMB, WMHD, BETASTAR, OMEGACYCL, ZEFF, PRAD, ENBI

In [122]:
DB5 = tokamakTK.clean_categorical_data(DB5)


# Needed to respectively clean each dtype
num_features = DB5[features].select_dtypes(include=['int', 'float']).columns.tolist()
cat_features = DB5[features].select_dtypes(include=['object']).columns.tolist()


data_num = DB5[num_features+["TOK","DATE"]]
data_cat = DB5[cat_features]

data_num = tokamakTK.clean_numerical_data(data_num)

data = data_num.copy()#pd.concat([data_num, tokamakTK.encode_categorical_ohe(data_cat)], axis=1)

## Correlation Matrix

A bivariate correlation of 0.8 or 0.9 is commonly used as a cut-ff to indicate a high correlation between two regressors. Nevertheless, high correlation does not imply multicollinearity, as they are not the same [[Chan, J. Y., et al, 2022]](https://doi.org/10.3390/math10081283). 

In [123]:
#sns.heatmap(data.corr(), cmap="vlag", annot=False);

## Variance Inflation Factor (VIF) and Tolerance

The Variation Inflation Factor (VIF) is a measure of multicollinearity in a linear regression model. It quantifies the degree to which the variance of the estimated regression coefficient is inflated due to collinearity in the predictors.

"One recommendation is that **if VIF is greater than 5**, then the explanatory variable given by exog_idx is highly collinear with the other explanatory variables, and the parameter estimates will have large standard errors because of this." [[statsmodels]](https://www.statsmodels.org/dev/generated/statsmodels.stats.outliers_influence.variance_inflation_factor.html)

$$
    \text{VIF}_j = \frac{1}{1-R^2_j}.
$$

With $R^2_j$ being the coefficient of determination for the regression of $x_j$ on the remaining variables; namely, 

$$
    R^2 = 1 - \frac{RSS}{TSS}.
$$

With $RSS$ being the sum of squares of residuals; and, $TSS$ the total sum of squares;

$$
    SSR = \sum_i\left( \hat{y} - \bar{y} \right)^2; \quad \text{and} \quad TSS = \sum_i\left( y - \bar{y} \right)^2
$$

**Tolerance** is the inverse of VIF and relates to the proportion of variance in a predictor variable. A **tolerance value less than 0.1** is generally considered to be indicative of problematic multicollinearity.

In [124]:
# VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = data.columns
  
# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(data.values, col_indx) for col_indx in range(len(data.columns))]
vif_data["Tolerance"] = 1/vif_data["VIF"]
vif_data = vif_data.T; vif_data.columns = vif_data.iloc[0]; vif_data.drop("feature", inplace=True)

vif_data

feature,BEIMHD,BT,DWDIA,ELMFREQ,EPS,IP,KAREA,LHTIME,MEFF,NEL,NESOL,NUSTAR,PECRH,PFLOSS,PICRH,PNBI,POHM,QCYL5,RHOSTAR,TAV,TIV,TORQ,WFFORM,WFICFORM,ZEFFNEO
VIF,3.097074,3.726706,2.474242,2.399748,2.883748,4.062999,3.251924,1.827179,1.500919,2.025249,2.813287,2.069517,1.251228,1.525013,1.309239,2.962955,2.352529,2.173886,3.779076,3.007297,2.55094,2.264708,2.659848,2.392605,1.756087
Tolerance,0.322885,0.268333,0.404164,0.41671,0.346771,0.246124,0.30751,0.547292,0.666259,0.493766,0.355456,0.483205,0.799215,0.655732,0.763803,0.337501,0.425074,0.460006,0.264615,0.332525,0.392012,0.441558,0.375961,0.417954,0.569448


In [125]:
vif_data.T[vif_data.T["VIF"] < 10 ]

,VIF,Tolerance
feature,,
BEIMHD,3.097074,0.322885
BT,3.726706,0.268333
DWDIA,2.474242,0.404164
ELMFREQ,2.399748,0.41671
EPS,2.883748,0.346771
IP,4.062999,0.246124
KAREA,3.251924,0.30751
LHTIME,1.827179,0.547292
MEFF,1.500919,0.666259


In [126]:
vif_data.T[vif_data.T["VIF"] >= 10 ]

,VIF,Tolerance
feature,,


In [120]:
vif_data.T[vif_data.T["VIF"] < 5 ].index

Index(['BEIMHD', 'BT', 'DWDIA', 'ELMFREQ', 'EPS', 'IP', 'KAREA', 'LHTIME',
       'MEFF', 'NEL', 'NESOL', 'NUSTAR', 'PECRH', 'PFLOSS', 'PICRH', 'PNBI',
       'POHM', 'QCYL5', 'RHOSTAR', 'TAV', 'TIV', 'TORQ', 'WFFORM', 'WFICFORM',
       'ZEFFNEO'],
      dtype='object', name='feature')

## Condition Index (or Condition Number)

Tthe ratio of the largest to the smallest eigenvalue of the correlation matrix. A condition number greater than 30 or 100 is often considered to be indicative of problematic multicollinearity.

In [ ]:
#1. Compute the covatiance matrix of the data
cov = np.cov(data, rowvar=False)
#2. Get the eigenvalues of the covariance matrix.
eigenvals, _ = np.linalg.eig(cov)
#3. Compute the condition number

cond_num = max(eigenvals) / min(eigenvals)
print('Condition number:', cond_num)

In this case, the condition number is sligtly greater than 5, which indicates that there may be some degree of multicollinearity in the dataset; but, not severe 

## Exploratory Data Analysis

In [ ]:
data["Label"] = DB5["label_str"]

g = sns.pairplot(data, 
             plot_kws=dict(marker="+", linewidth=1),
             diag_kws=dict(fill=False),
             hue="Label",
             palette="Set2"
            );
for ax in g.axes.flat:
    ax.set_ylabel(ax.get_ylabel(), rotation = 0)
    ax.set_xlabel(ax.get_xlabel(), rotation = 45)
    ax.yaxis.set_label_coords(-3, 0.5)
    ax.xaxis.set_label_coords(0.3, -1.5)
sns.move_legend(g, "upper left", bbox_to_anchor=(1, 1))
g.fig.set_size_inches(7,5)

## Principal Component Analysis (PCA)

In [ ]:
#1. Standarize data: this is already done by tokamakTK
data = data.drop("Label", axis="columns")

#2. Perform the PCA
pca = PCA()
pca.fit(data)

#3. Examine the variance explained by each PC
variance_ratio = pca.explained_variance_ratio_
cumulative_variance_ratio = np.cumsum(variance_ratio)

In [ ]:
# Print the results
print("Variance explained by each PC:\n")
pd.DataFrame([variance_ratio], columns=data.columns, index=["Variance"]).T

In [ ]:
print("Cumulative variance explained by each PC:\n")
pd.DataFrame([cumulative_variance_ratio], columns=data.columns, index=["Cum Variance"]).T